<a href="https://colab.research.google.com/github/keysun8/videos/blob/main/yt_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["OPENAI_API_KEY"] =

In [ ]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv

In [ ]:
pip install pinecone-client

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI()

In [ ]:
llm.invoke("who is Demis Hassabis ? ")

AIMessage(content='Demis Hassabis is a British artificial intelligence researcher, neuroscientist, and entrepreneur. He is the co-founder and CEO of DeepMind, a London-based artificial intelligence research company that was acquired by Google in 2014. Hassabis has a background in computer science, neuroscience, and artificial intelligence, and has made significant contributions to the field of AI research.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 15, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BWzq5OpiFLinbGW8n315q8NzP17Sq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e5f5c0c0-f22e-45bf-b014-a106cfa18521-0', usage_metad

In [ ]:
import re

def extract_youtube_id(url: str) -> str:
    # Regex to handle multiple YouTube URL formats
    pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11}).*'
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    else:
        raise ValueError("Invalid YouTube URL")


In [ ]:
url = "https://www.youtube.com/watch?v=TcmGYe39XG0&list=PLBD53FA93C10327B9"
video_id = extract_youtube_id(url)
print(video_id)  # Output: dQw4w9WgXcQ
# video_id = "Xj0Jtjg3lHQ" # only the ID, not full URL
try:
    # If you don’t care which language, this returns the “best” one
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

    # Flatten it to plain text
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")

TcmGYe39XG0
We take off now from last semester where we
left off with quantum physics this semester. Before we begin the course proper, I would
like to get a little background from you on what you already know, how we should run this
course, what the modalities are and so on. As far as the evaluation is concerned, it's
the usual two quizzes and a final exam. Philosophical and interpretational aspects of quantum mechanics
are still open in the sense that, all questions have not been answered. In spite of over a
hundred years of effort, there are unresolved questions, there are deep mysteries, and there
are central mysteries and issues. Some of them may turn out in the future to be non-questions
in the sense that, you yourself may turn out to be asking the wrong question and so on.
But at the operational level, it's actually easier than classical physics. The mathematics
is simpler than classical mechanics, for instance, and it works over a huge wide range of phenomena. As far as we know

In [ ]:
# video_id = "Xj0Jtjg3lHQ" # only the ID, not full URL
# try:
#     # If you don’t care which language, this returns the “best” one
#     transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

#     # Flatten it to plain text
#     transcript = " ".join(chunk["text"] for chunk in transcript_list)
#     print(transcript)

# except TranscriptsDisabled:
#     print("No captions available for this video.")

In [ ]:
# transcript_list

Step 1b - Indexing (Text Splitting)

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [ ]:
len(chunks)

56

In [ ]:
chunks[10]

Document(metadata={}, page_content="an open problem. The question of what happens in open systems which exchange energy and\nmomentum and so on with the surroundings is still an open problem. In that sense, the\nquantum mechanics we are going to do is really very elementary for systems which are essentially\ngoing to be at absolute zero of temperature. So we won't bring in temperature and systems\nwhich don't have any dissipation at all, till fairly late in the course. So it's\nfor those kinds of coordinates in momentum that you have the uncertainty principle. Now\nwhat does the uncertainty principle actually say? Let's look at x and px for a particle. This\nis a canonically conjugate pair. Classically I know that this is equal to 1. In the sense\nof Poisson brackets, this is 1 if I look at a single particle and I say x is a generalized\ncoordinate and px is a corresponding conjugate momentum, the Poisson bracket of x with px\nis 1. Now there is a similar relation written down quantum 

Sementic memory based(experminal based)

In [ ]:
pip install -U langchain langchain-experimental sentence-transformers


In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

In [ ]:
sementic_text_splitter = SemanticChunker(
    OpenAIEmbeddings(model="text-embedding-3-small"),
    breakpoint_threshold_type="standard_deviation",
    breakpoint_threshold_amount=10
)


In [ ]:
docs = sementic_text_splitter.create_documents([transcript])

In [ ]:
print(len(docs))

1


Embedding Generation and Storing in Vector Store

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
vector_store = FAISS.from_documents(chunks, embeddings)

In [ ]:
vector_store.index_to_docstore_id

{0: 'ca1547d9-c423-4b57-b2e5-c44a18d24f2e',
 1: 'a04c70f0-3ed5-4f32-9e22-b5abf85e62e6',
 2: '9c5d6c7c-221c-4f7f-9338-dcc1e9caa477',
 3: '8806e0db-924c-4d91-a3e3-62a455374199',
 4: '6c4cf988-c047-4b85-ab31-6e1f0cb1cef8',
 5: 'e9ff7ce5-63e3-4163-956b-60e3ebdcfa61',
 6: '5ee57e89-9db6-493d-8ff9-2862cd81788a',
 7: '2821462c-e529-4109-a88f-813829de3f8d',
 8: '53114ce7-f75a-4cc3-a9f0-b1e4332bc8a9',
 9: 'dfeea45d-0108-444e-8f88-b002d9b7696e',
 10: 'e1fc1b6f-d460-4153-a34c-e48f87de63b2',
 11: '8595dce8-438a-484f-87e4-3a13c03202ac',
 12: '8434025e-a428-47b7-8a7b-1a28c156be83',
 13: '460fec04-abf0-4c83-b2bf-4c16385ad413',
 14: '6ffb7cdb-f643-468a-92bd-105f77c56f4b',
 15: '975670bb-e03f-4c9c-aeae-6cb266605e03',
 16: 'ad3633c8-1753-4bc4-827c-44a5f9d95130',
 17: '30b683ed-504c-44cc-a69e-0fd2f897558c',
 18: '7a70538a-215f-4156-9cba-d614bdc9016d',
 19: 'c188acc7-4e88-4f02-a115-1b93e0e39b6f',
 20: '8e810faf-6807-4817-a25c-84164a9f0a18',
 21: '0dc17ed0-0aa9-40d8-8418-0952366049e9',
 22: 'bc217d7f-4a07-

Step 2 - Retrieval

In [ ]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 3, "fetch_k": 50, "lambda_mult": 0.5})

In [ ]:
retriever.invoke('What is burj khalifa?')

[Document(id='00999f74-d783-48fe-a35a-19345c28b9ac', metadata={}, page_content="that our very ideas of space- time, what we mean by dimension, etc, as well as what we\nmean by normal quantum mechanics I think, will undergo drastic revision. So it's a\nhouse under construction. It looks like the existing frame work is not powerful enough\nto do this. You could ask now what about experiment. That\nwe lag far behind. The shortest distances we can probe are of the order of ten to the\npower minus seventeen or eighteen. This is all we have been able to probe so far in the\nhighest energy accelerators. There are sixteen orders of magnitude or seventeen orders of\nmagnitude between these two. We don't know what hat can happen in there. Similarly, the shortest length scales we have\nbeen able to probe which are of the order of 10 to the -- 23 seconds. There are twenty\norders of magnitude in between. We don't know how things will change. So we keep an\nopen mind completely. That's the whole po

Step 3 - Augmentation

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

In [ ]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [ ]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [ ]:
retrieved_docs

[Document(id='3c650d66-2eec-4f6f-a68d-4c32a58e4df7', metadata={}, page_content="and the way it's set up is not satisfying. So it already looks like there are big black\nboxes in it which we have pushed under the rug but they exist nevertheless. We have not\nunderstood quantum non-locality. Except in some mathematical form of a framework, we\nprove our satisfaction. It really hasn't come to grips with the meaning of measurement.\nNow what happens to a system when you measure it? So there are large gaps conceptual gaps\nhere. So it's clear it's an incomplete theory in that sense at least that understanding\nis not as deep as we would like it to be the framework of quantum mechanics. You could\nask may be this is the best you can do how do you know we don't know that too we don't\nknow but we have to keep pushing so today has been a sort of general talk. We will start\nof with some specific issues and so on next time."),
 Document(id='647cd75b-7bfb-4098-952b-5173c046a481', metadata={}, pa

In [ ]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"and the way it's set up is not satisfying. So it already looks like there are big black\nboxes in it which we have pushed under the rug but they exist nevertheless. We have not\nunderstood quantum non-locality. Except in some mathematical form of a framework, we\nprove our satisfaction. It really hasn't come to grips with the meaning of measurement.\nNow what happens to a system when you measure it? So there are large gaps conceptual gaps\nhere. So it's clear it's an incomplete theory in that sense at least that understanding\nis not as deep as we would like it to be the framework of quantum mechanics. You could\nask may be this is the best you can do how do you know we don't know that too we don't\nknow but we have to keep pushing so today has been a sort of general talk. We will start\nof with some specific issues and so on next time.\n\norbit. That cannot be true. Because if that were true, you have this particle\nhere with the center of attraction O and something moving at the Boh

In [ ]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [ ]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      and the way it's set up is not satisfying. So it already looks like there are big black\nboxes in it which we have pushed under the rug but they exist nevertheless. We have not\nunderstood quantum non-locality. Except in some mathematical form of a framework, we\nprove our satisfaction. It really hasn't come to grips with the meaning of measurement.\nNow what happens to a system when you measure it? So there are large gaps conceptual gaps\nhere. So it's clear it's an incomplete theory in that sense at least that understanding\nis not as deep as we would like it to be the framework of quantum mechanics. You could\nask may be this is the best you can do how do you know we don't know that too we don't\nknow but we have to keep pushing so today has been a sort of general talk. We will start\nof with so

Step 4 - Generation

In [ ]:
answer = llm.invoke(final_prompt)
print(answer.content)

I don't know.


In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [ ]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [ ]:
parser = StrOutputParser()

In [ ]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke('does they talk about the facebook ? ')

"I don't know."

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage , AIMessage
chat_history = [
    SystemMessage(content = "You are a helpful coder"),
]

In [ ]:
print("Ask your questions below. Type 'exit' to stop.\n")

while True:
    question = input("You: ")
    if question.lower() in ["exit", "quit"]:
        break
    result = main_chain.invoke(question)
    print(f"AI: {result}\n")


Ask your questions below. Type 'exit' to stop.

AI: The mathematical formulation of Heisenberg's uncertainty principle states that the product of the uncertainties in a pair of canonically conjugate variables, such as position (Δr) and momentum (Δpr), is greater than or equal to a constant. Specifically, it can be expressed as:

Δr * Δpr ≥ ℏ / 2

where ℏ (h cross) is the reduced Planck's constant. This principle indicates that you cannot simultaneously measure both variables with arbitrary precision.

